In [ ]:
import math
import numpy as np
import cv2

# Function to calculate SSIM (Structural Similarity Index) between two images
def ssim(img1, img2):
    # Constants for SSIM calculation
    C1 = (0.01 * 255)**2
    C2 = (0.03 * 255)**2

    # Convert images to float64 for precision in calculations
    img1 = img1.astype(np.float64)
    img2 = img2.astype(np.float64)

    # Create a Gaussian kernel for convolution
    kernel = cv2.getGaussianKernel(11, 1.5)
    window = np.outer(kernel, kernel.transpose())

    # Calculate means using Gaussian filter
    mu1 = cv2.filter2D(img1, -1, window)[5:-5, 5:-5]  # valid region
    mu2 = cv2.filter2D(img2, -1, window)[5:-5, 5:-5]
    mu1_sq = mu1**2
    mu2_sq = mu2**2
    mu1_mu2 = mu1 * mu2

    # Calculate variances and covariance
    sigma1_sq = cv2.filter2D(img1**2, -1, window)[5:-5, 5:-5] - mu1_sq
    sigma2_sq = cv2.filter2D(img2**2, -1, window)[5:-5, 5:-5] - mu2_sq
    sigma12 = cv2.filter2D(img1 * img2, -1, window)[5:-5, 5:-5] - mu1_mu2

    # Calculate SSIM map
    ssim_map = ((2 * mu1_mu2 + C1) * (2 * sigma12 + C2)) / ((mu1_sq + mu2_sq + C1) * (sigma1_sq + sigma2_sq + C2))
    return ssim_map.mean()  # Return mean SSIM value

# Function to calculate SSIM for grayscale and RGB images
def calculate_ssim(img1, img2):
    # Check if input images have the same dimensions
    if not img1.shape == img2.shape:
        raise ValueError('Input images must have the same dimensions.')

    # If grayscale image
    if img1.ndim == 2:
        return ssim(img1, img2)
    # If RGB image
    elif img1.ndim == 3:
        if img1.shape[2] == 3:  # RGB channels
            ssims = []
            for i in range(3):
                ssims.append(ssim(img1, img2))
            return np.array(ssims).mean()  # Return mean SSIM across channels
        elif img1.shape[2] == 1:  # Single channel image
            return ssim(np.squeeze(img1), np.squeeze(img2))
    else:
        raise ValueError('Wrong input image dimensions.')

# Main function to load images and calculate SSIM value
def main():
    # Load original and compressed images
    original = cv2.imread("image1.jpg")
    compressed = cv2.imread("output_image.jpg", 1)

    # Calculate SSIM value
    value = calculate_ssim(original, compressed)
    print(f"SSIM value is {value} dB")

if __name__ == "__main__":
    main()
